<a href="https://colab.research.google.com/github/Dicere/Machine_Learning_algorithms_from_scratch/blob/main/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import random

In [5]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt'
df = pd.read_csv(url, header=None)
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
X, y = df.iloc[:,:4], df['target']

In [148]:
class MyLogReg():
    def __init__(self, n_iter=100, learning_rate=0.1, weights=None, metric=None, verbose=1, score=0, reg=None, l1_coef=0, l2_coef=0,sgd_sample=None,random_state=42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.verbose = verbose
        self.metric = metric
        self.score = score
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample  = sgd_sample
        self.random_state  = random_state
    def __str__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLogReg class: {attributes}"
    def __repr__(self):
        attributes = ', '.join(f"{key}={value}" for key, value in vars(self).items())
        return f"MyLogReg class: {attributes}"

    def fit(self, X_fit: pd.DataFrame, y_fit: pd.Series, n_iter=None, lr=None, verbose=None):
      X = X_fit.copy()
      y = y_fit.copy()
      X.insert(0, "One", 1)
      n = X.shape[0]
      self.weights =  np.ones(X.shape[1])

      if n_iter is not None:
          self.n_iter = n_iter
      if lr is not None:
          self.learning_rate = lr
      if verbose is not None:
          self.verbose = verbose

      for i in range(self.n_iter):
        
        y_pred_vec = 1 / (1+ np.exp(-1*(X.to_numpy() @ self.weights)))
        
        y_without_null = np.clip(y_pred_vec, 1e-15, 1 - 1e-15)
        LogLoss = -1/n*sum(y*np.log(y_without_null)+(1-y)*np.log(1-y_without_null))
        err=np.subtract(y_pred_vec,y)
        grad_vec = (1 / X.shape[0]) * (err @ (X.to_numpy()))
        self.weights -= self.learning_rate * grad_vec

        if i !=0 and self.verbose != 0:
          if i % self.verbose == 0:
            print(f"{i}:{LogLoss}")

    def get_coef(self):
        return self.weights[1:]

    def predict(self, X_test: pd.DataFrame):
      X = X_test.copy()
      X.insert(0, "One", 1)
      y_pred_vec = 1 / (1+ np.exp(-1*(X.to_numpy() @ self.weights)))
      res = np.where( y_pred_vec > 0.5, 1, 0 ) 
      return res
      
    def predict_proba(self, X_test: pd.DataFrame):
      X = X_test.copy()
      X.insert(0, "One", 1)
      y_pred_vec = 1 / (1+ np.exp(-1*(X.to_numpy() @ self.weights)))
      return y_pred_vec

In [149]:
instance = MyLogReg()

In [150]:
instance.fit(X,y,n_iter=100,lr=0.01,verbose=10)

10:3.85319982029842
20:3.202352848896302
30:2.68812810953371
40:2.2756150117046396
50:1.9250538326422728
60:1.6175146325825533
70:1.3471696849782449
80:1.1140696508933912
90:0.9199630466635381


In [153]:
instance.predict_proba(X)

array([0.3590127 , 0.16658887, 0.44660179, ..., 0.92578526, 0.94827984,
       0.95349749])